In [16]:
import numpy as np

In [17]:
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        funcs = [self.creator]
        while funcs:
            f = funcs.pop()             # 함수를 가져온다.
            x, y = f.input, f.output    # 함수의 입력과 출력을 가져온다.
            x.grad = f.backward(y.grad) # backward 메서드를 호출한다.

            if x.creator is not None:
                funcs.append(x.creator)

In [18]:
class Function:
    def __call__(self, input):
        x = input.data           # 데이터를 꺼낸다.
        y = self.forward(x)      # 구체적인 계산은 forward 메서드에서 한다.
        output = Variable(y)     # Variable 형태로 되돌린다.
        output.set_creator(self) # 출력 변수에 창조자를 설정한다.
        self.input = input       # 입력 변수를 기억(보관)한다.
        self.output = output     # 출력도 저장한다.
        return output

    def forward(self, x):
        raise NotImplementedError()

    def backward(self, gy):
        raise NotImplementedError()

In [19]:
class Square(Function):
    def forward(self, x):
        return x ** 2

    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx

class Exp(Function):
    def forward(self, x):
        return np.exp(x)

    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx

In [20]:
x = Variable(np.array(0.5))
f = Square()
y = f(x)

In [21]:
def square(x):
    f = Square()
    return f(x)

def exp(x):
    f = Exp()
    return f(x)

In [22]:
x = Variable(np.array(0.5))
a = square(x)
b = exp(a)
y = square(b)

# 역전파
y.grad = np.array(1.0)
y.backward()
print(x.grad)

3.297442541400256


In [23]:
x = Variable(np.array(0.5))
y = square(exp(square(x))) # 연속하여 적용

# 역전파
y.grad = np.array(1.0)
y.backward()
print(x.grad)

3.297442541400256


In [25]:
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()             # 함수를 가져온다.
            x, y = f.input, f.output    # 함수의 입력과 출력을 가져온다.
            x.grad = f.backward(y.grad) # backward 메서드를 호출한다.

            if x.creator is not None:
                funcs.append(x.creator)

In [29]:
x = Variable(np.array(0.5))
y = square(exp(square(x))) # 연속하여 적용
y.backward()
print(x.grad)

3.297442541400256


In [30]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError(f'{type(data)}은(는) 지원하지 않습니다.')

        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        funcs = [self.creator]
        while funcs:
            f = funcs.pop()             # 함수를 가져온다.
            x, y = f.input, f.output    # 함수의 입력과 출력을 가져온다.
            x.grad = f.backward(y.grad) # backward 메서드를 호출한다.

            if x.creator is not None:
                funcs.append(x.creator)

In [36]:
x = Variable(np.array(1.0)) # OK
x = Variable(None)          # OK

x = Variable(1.0)           # TypeError: <class 'float'>은(는) 지원하지 않습니다.

TypeError: <class 'float'>은(는) 지원하지 않습니다.

In [38]:
x = np.array([1.0])
y = x ** 2
print(type(x), x.ndim)
print(type(y))

<class 'numpy.ndarray'> 1
<class 'numpy.ndarray'>


In [39]:
x = np.array(1.0) # 0차원 ndarray
y = x ** 2
print(type(x), x.ndim)
print(type(y))

<class 'numpy.ndarray'> 0
<class 'numpy.float64'>


In [ ]:
def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

In [ ]:
class Function:
    def __call__(self, input):
        x = input.data                 # 데이터를 꺼낸다.
        y = self.forward(x)            # 구체적인 계산은 forward 메서드에서 한다.
        output = Variable(as_array(y)) # Variable 형태로 되돌린다.
        output.set_creator(self)       # 출력 변수에 창조자를 설정한다.
        self.input = input             # 입력 변수를 기억(보관)한다.
        self.output = output           # 출력도 저장한다.
        return output

    def forward(self, x):
        raise NotImplementedError()

    def backward(self, gy):
        raise NotImplementedError()